**Note：此文件用对8个novel对象的测试集进行评测**

计算指标,如遇问题请进入MyEval.py文件修改

In [1]:
! python -m torch.distributed.run --nproc_per_node=4 MyEval.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 3, word 4): env://eval
| distributed init (rank 1, word 4): env://eval
| distributed init (rank 2, word 4): env://eval
| distributed init (rank 0, word 4): env://eval
Creating model..
use my model!
1..
2..
load checkpoint from output/Caption_coco_DCC_pretrain_1/checkpoint_best.pth
3..
model created!
Creating captioning dataset
Start evaluating  captions_split_set_bottle_val_test_novel2014.json ..
Caption generation:  [ 0/10]  eta: 0:01:42    time: 10.2136  data: 2.1445  max mem: 3037
Caption generation:  [ 9/10]  eta: 0:00:03    time: 3.6274  data: 0.2146  max mem: 3037
Caption generation: Total time: 0:00:36 (3.6458 s / it)
Traceback (most recent call last):
Traceback (mo

F1得分计算

In [2]:
from pycocotools.coco import COCO
import json
from json import encoder
import pandas as pd
encoder.FLOAT_REPR = lambda o: format(o, '.3f')

def eval_f1(keyword,pred,gt):
    tp=0;fp=0;fn=0
    df1=pd.DataFrame(pred)
    df2=pd.DataFrame(gt)
    for index in range(len(df1)):
        id=df1.loc[index].image_id
        pred_cap=df1.loc[index].caption
        pred_positive=True if pred_cap.find(keyword)!= -1 else False
        temp=df2[df2.image_id==id]
        len1=len(temp)
        temp=temp[temp.caption.str.contains(keyword)]
        if pred_positive and len(temp)==len1: tp+=1
        if pred_positive and len(temp)!=len1: fp+=1
        if not pred_positive and len(temp)==len1: fn+=1
    
    print('-'*5,'keyword:',keyword,'-'*5)
    print('tp: %d, fp: %d, fn: %d, F1_score: %.3f'%(tp,fp,fn,tp/(tp+0.5*fp+0.5*fn)))
    print('-'*25) 


ann_file="annotation/DCC/captions_val_test2014.json" 
# ground_truth
annFile= ann_file
coco = COCO(annFile)
gt = coco.dataset['annotations']
group1=['bottle','bus','couch','microwave','pizza','racket','suitcase','zebra']
group2=['couch','microwave']

for name in group2:
    pred_file='output/DCC_eval/result/%s_predict.json' %name
    # prediction
    with open(pred_file, 'rb') as f:
        pred = json.load(f)
    eval_f1(name,pred,gt)

loading annotations into memory...
Done (t=3.80s)
creating index...
index created!
----- keyword: couch -----
tp: 22, fp: 106, fn: 9, F1_score: 0.277
-------------------------
----- keyword: microwave -----
tp: 16, fp: 24, fn: 2, F1_score: 0.552
-------------------------
